In [ ]:
import os

In [ ]:
os.makedirs('/content/drive/MyDrive/heat_backup', exist_ok=True)

In [ ]:
import tqdm
import json
import math
import torch
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import eigs


class GaussianRF(object):

    def __init__(self, dim, size, alpha=2, tau=3, sigma=None, boundary="periodic", device=None):

        self.dim = dim
        self.device = device

        if sigma is None:
            sigma = tau**(0.5*(2*alpha - self.dim))

        k_max = size//2

        if dim == 1:
            k = torch.cat((torch.arange(start=0, end=k_max, step=1, device=device), \
                           torch.arange(start=-k_max, end=0, step=1, device=device)), 0)

            self.sqrt_eig = size*math.sqrt(2.0)*sigma*((4*(math.pi**2)*(k**2) + tau**2)**(-alpha/2.0))
            self.sqrt_eig[0] = 0.0

        elif dim == 2:
            wavenumers = torch.cat((torch.arange(start=0, end=k_max, step=1, device=device), \
                                    torch.arange(start=-k_max, end=0, step=1, device=device)), 0).repeat(size,1)

            k_x = wavenumers.transpose(0,1)
            k_y = wavenumers

            self.sqrt_eig = (size**2)*math.sqrt(2.0)*sigma*((4*(math.pi**2)*(k_x**2 + k_y**2) + tau**2)**(-alpha/2.0))
            self.sqrt_eig[0,0] = 0.0

        elif dim == 3:
            wavenumers = torch.cat((torch.arange(start=0, end=k_max, step=1, device=device), \
                                    torch.arange(start=-k_max, end=0, step=1, device=device)), 0).repeat(size,size,1)

            k_x = wavenumers.transpose(1,2)
            k_y = wavenumers
            k_z = wavenumers.transpose(0,2)

            self.sqrt_eig = (size**3)*math.sqrt(2.0)*sigma*((4*(math.pi**2)*(k_x**2 + k_y**2 + k_z**2) + tau**2)**(-alpha/2.0))
            self.sqrt_eig[0,0,0] = 0.0

        self.size = []
        for j in range(self.dim):
            self.size.append(size)

        self.size = tuple(self.size)

    def sample(self, N):

        coeff = torch.randn(N, *self.size, dtype=torch.cfloat, device=self.device)
        coeff = self.sqrt_eig * coeff

        return torch.fft.ifftn(coeff, dim=list(range(-1, -self.dim - 1, -1))).real

from data_work import build_matrix

In [ ]:
from google.colab import drive
import time, os, shutil
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

# while True:
#     # Find .pt files in /content
#     files = [f for f in os.listdir('/content') if f.endswith('.pt')]
#     for f in files:
#         src = f'/content/{f}'
#         dst = f'/content/drive/MyDrive/heat_backup/{f}'
#         if not os.path.exists(dst):
#             shutil.copy(src, dst)
#             print(f"✅ Сохранено: {f}")

#     time.sleep(300)  # 5 минут


In [ ]:
s = 302; k = 25; edge = s-2; tol = 1e-8; alpha = 1
train_num, test_num = 3000, 500
batch_size = 100  # ✅ 4.8 ГБ пик

# Убираем большие w/p заранее!
GRF = GaussianRF(2, s, alpha=2, tau=3, device='cpu')
x, eig_vecs, eig_vals = [], [], []

pbar = tqdm.tqdm(total=train_num+test_num, desc=f"Heat Resolution{s}")

batch_num = 0
for i in range(train_num + test_num):
    pbar.update()

    # Генерируем ПО ОДНОМУ (не 3501 сразу!)
    w_i = np.exp(GRF.sample(1))[0] * alpha
    p_i = torch.empty(1, edge, edge).uniform_(-1, 1)
    pi = torch.abs(p_i[0])**2

    A1 = build_matrix(w_i, pi)
    eig_val, eig_vec = eigs(A1, k, sigma=0, which="LM", tol=tol)

    # Padding pi
    pi_pad = torch.cat([torch.zeros(edge,1), pi, torch.zeros(edge,1)], dim=-1)
    pi_pad = torch.cat([torch.zeros(1,s), pi_pad, torch.zeros(1,s)], dim=0)

    x.append(torch.cat((w_i.unsqueeze(2), pi_pad.T.unsqueeze(2)), dim=-1).unsqueeze(0))
    eig_vecs.append(torch.tensor(eig_vec.real, dtype=torch.float32).unsqueeze(0))
    eig_vals.append(torch.tensor(eig_val, dtype=torch.float32).unsqueeze(0))

    # ✅ СОХРАНЯЕМ БАТЧ
    if len(x) >= batch_size:
        batch_data = {
            "params": torch.cat(x, dim=0),
            "eig_vecs": torch.cat(eig_vecs, dim=0),
            "eig_vals": torch.cat(eig_vals, dim=0)
        }
        torch.save(batch_data, f"heat_batch_{batch_num}.pt")
        src = f'/content/heat_batch_{batch_num}.pt'
        dst = f'/content/drive/MyDrive/heat_backup/heat_batch_{batch_num}.pt'
        if not os.path.exists(dst):
            shutil.copy(src, dst)



        x, eig_vecs, eig_vals = [], [], []
        batch_num += 1
        import gc; gc.collect()

pbar.close()



Heat Resolution302:  11%|█▏        | 400/3500 [40:38<5:14:58,  6.10s/it]

Heat Resolution302:  69%|██████▉   | 2431/3500 [3:31:49<1:45:14,  5.91s/it]